In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import catboost
from catboost import Pool
import xgboost
import optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv('/kaggle/input/brasilian-houses-to-rent/houses_to_rent_v2.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
#df.columns = ['city', 'area', 'rooms', 'bathroom', 'parking spaces', 'floor',
#       'animal', 'furniture', 'hoa', 'rent amount',
#       'property tax', 'fire insurance', 'total']

In [ ]:
def categorise(data,cols):
    object_cols = []
    num_cols = []
    for i in cols:
        if data[i].dtype == 'object':
            object_cols.append(i)
        else:
            num_cols.append(i)
    return object_cols, num_cols

In [ ]:
object_cols, num_cols = categorise(df,df.columns)
object_cols, num_cols

So, four columns i.e. 'city', 'floor', 'animal', 'furniture' have values other than a numerical one. Now, let's first check if the data contains any NaN values.

In [ ]:
df.isnull().sum()

Clearly, data not contains any NaN values. Let's check what all different values the object_cols has.

In [ ]:
df['city'].unique()

Since, 'city' column contains 5 different values, so we'll use one_hot_encorder.

In [ ]:
df['floor'].unique()

'floor' contains '-', it has to be removed. 

In [ ]:
df.loc[df['floor'] == '-', 'floor'] = 0
df['floor'] = df['floor'].astype('int64')

In [ ]:
df['animal'].unique()

In [ ]:
df['furniture'].unique()

# Checking for outliers

In [ ]:
sns.boxplot(data=df['rent amount (R$)'])

**We can see that there are some outliers, so we have to remove them**

# Removing outliers

In [ ]:
df1 = df.copy()

In [ ]:
df1.head()

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [ ]:
df2 = remove_outlier(df1, 'rent amount (R$)')

In [ ]:
df2.shape

In [ ]:
sns.boxplot(data=df['rent amount (R$)'])

In [ ]:
# Lets take a look on how our data was distributed before and after treat outliers

plt.figure(1, figsize=(20, 10))
plt.subplot(2, 2, 1)
sns.distplot(df['rent amount (R$)'])
plt.title('Before Removing Outliers')
plt.subplot(2, 2, 2)
sns.distplot(df2['rent amount (R$)'])
plt.title('After Removing Outliers')

In [ ]:
num_cols1 = ['rooms', 'bathroom', 'parking spaces']
plt.figure(figsize=(20, 5))
sns.set(style = 'whitegrid')
i = 1
for feature in num_cols1:
    plt.subplot(2, 3, i)
    sns.barplot(x = feature, y= 'rent amount (R$)', data=df2)
    i+=1
plt.tight_layout()

In [ ]:
# now let's see the correlation between features
plt.figure(figsize=(12,12))
sns.heatmap(df2.corr(), annot=True, cmap='RdBu_r', linecolor='black',vmin=-1, vmax=1)

In [ ]:
lev_enc = LabelEncoder()
df2['animal']=lev_enc.fit_transform(df2['animal'])
df2['furniture']=lev_enc.fit_transform(df2['furniture'])

In [ ]:
df2.head()

In [ ]:
hot_enc = pd.get_dummies(df2['city'])

In [ ]:
df_final = pd.concat([df2, hot_enc], axis=1)

In [ ]:
df_final.drop(['city'], axis=1, inplace=True)

In [ ]:
df_final.head(10)

In [ ]:
columns = ['Belo Horizonte','Campinas', 'Porto Alegre', 'Rio de Janeiro', 'São Paulo', 'rooms', 'bathroom', 'parking spaces', 'fire insurance (R$)',
        'furniture']
X = df_final[columns]
y = df_final['rent amount (R$)']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, random_state=0)

In [ ]:
X.info()

In [ ]:
train_pool = Pool(
                data = X_train,
                label = y_train,
)
valid_pool = Pool(
                data = X_valid,
                label = y_valid,
)

In [ ]:
model = catboost.CatBoostRegressor(custom_metric= ['R2', 'RMSE'], learning_rate=0.1, n_estimators=264)
model.fit(train_pool, eval_set=valid_pool, verbose=50, plot=True)

In [ ]:
# we create a list to storage all the results for later visualization
acc = []
# parameters are the alpha's that we will use to perform the GridSearch
parameters1= [{'alpha': [0.0001, 0.001, 0.1, 1, 10, 100, 1000, 10000, 100000, 100000]}]
# on the regressors we define the models that we want use
regressors = {'Linear Regression': LinearRegression(),
              'Ridge Model': Ridge(alpha=0.1),
              'Decision Tree': DecisionTreeRegressor(),
              'Random Forest': RandomForestRegressor(random_state=1),
              'SVR': SVR(),
              'KNN': KNeighborsRegressor(),
              'Lasso': Lasso(),
              'GridSearchRidge': GridSearchCV(Ridge(), parameters1, cv=4),
              'GridSearchLasso': GridSearchCV(Lasso(), parameters1, cv=4)
             }

In [ ]:
# now we perform a loop with each regressor to perform the model, predict the rent 
# and extract the metrics
for i in regressors:
    model = regressors.get(i)
    # here we create a condition because for grid we want to perform the model with the best estimator
    if i == 'GridSearchRidge' or i == 'GridSearchLasso':
        model.fit(X_train, y_train).best_estimator_ 
    model.fit(X_train, y_train)
    prediction = model.predict(X_valid)
    print(i)
    print('MAE:', mean_absolute_error(y_valid, prediction))
    print('RMSE:', np.sqrt(mean_squared_error(y_valid, prediction)))
    print('R2:', r2_score(y_valid, prediction))
    print('*' * 40)
    acc.append([i, mean_absolute_error(y_valid, prediction), np.sqrt(mean_squared_error(y_valid, prediction)), r2_score(y_valid, prediction)])


In [ ]:
# now let's follow the same loop and visualize the plot's for each regressor
j = 1
plt.figure(figsize=(20,10))
for i in regressors:
    model = regressors.get(i)
    model.fit(X_train, y_train)
    prediction = model.predict(X_valid)
    plt.subplot(3, 3, j)
    plt.title(i)
    ax1 = sns.distplot(y_valid,hist=False,kde =True,color ="r",label ="Actual Value")
    sns.distplot(prediction ,color ="b",hist = False,kde =True, label = "Predicted Value",ax =ax1).set_title(i)
    j+=1
plt.tight_layout(pad = 0.5)